In [56]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import time
from itertools import islice
from tqdm import tqdm
import json
load_dotenv()

BASE_URL_AB = 'https://acousticbrainz.org'
BASE_URL_MB = 'https://musicbrainz.org/ws/2'
lastfm_tracks = pd.read_csv('data/lastfm/tracks_df.csv')
lastfm_tracks_mbids = lastfm_tracks['mbid'].unique()

# Rate limit of 1 request per second is advised by the website

In [11]:
endpoint = f"{BASE_URL_AB}/api/v1/high-level"

ab_data = {}

batch_size = 25
for i in tqdm(range(0, len(lastfm_tracks_mbids), batch_size), desc="Processing batches"):
    batch = lastfm_tracks_mbids[i:i + batch_size]
    params = {
        'recording_ids': ';'.join(batch),
    }

    response = requests.get(endpoint, params=params)
    ab_data.update(response.json())



Processing batches: 100%|██████████| 181/181 [04:56<00:00,  1.64s/it]


In [13]:
with open('data/brainz/ab_data_raw.json', 'w') as f:
    json.dump(ab_data, f)

In [40]:
lookups = ';'.join(list(ab_data.keys())[:10])

In [79]:

mb_endpoint = f"https://musicbrainz.org/ws/2/recording/"
params = {
        "query": f"mbid:{mbid_query}",
        "fmt": "json"
    }
headers = {"User-Agent": "MyMusicApp/1.0 (your-email@example.com)"}
    
response = requests.get(mb_endpoint, params=params,headers=headers)

In [142]:
endpoint = f"{BASE_URL_MB}/recording/"

mb_data = []

# Process in batches to avoid too large requests
batch_size = 25  # MusicBrainz typically has limits on request sizes
for i in tqdm(range(0, len(list(ab_data.keys())), batch_size), desc="Processing MBIDs"):
    batch = list(ab_data.keys())[i:i + batch_size]
    
    # Join MBIDs with OR operator
    mbid_query = " OR ".join([f"mbid:{mbid}" for mbid in batch])
    
    params = {
        "query": mbid_query,
        "fmt": "json"
    }

    headers = {"User-Agent": "JHU-Project/1.0 (spalit2@jh.edu)"}

    response = requests.get(endpoint, params=params, headers=headers)

    mb_data.extend(response.json()['recordings'])

    time.sleep(0.3)


Processing MBIDs: 100%|██████████| 177/177 [03:36<00:00,  1.22s/it]


In [153]:
mb_df = pd.DataFrame(mb_data)
mb_df.to_csv('data/brainz/mb_data.csv', index=False)

In [152]:
len(mb_df[mb_df['isrcs'].notna()])

2457